In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# n = 0
# for dirname, _, filenames in os.walk('/kaggle/input/'):
#     for filename in filenames:
#         if dirname == '/kaggle/input/petfinder-pawpularity-score/test' or dirname == '/kaggle/input/petfinder-pawpularity-score/train':
#             break
#         else:
#             print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
sdsd

In [ ]:
import wandb

from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
wandb.init(project='PetFinder', entity='kudasov_dmitriy')

In [ ]:
import sys
sys.path.append("../input/timm-folder/")

In [ ]:
TRAIN = pd.read_csv('/kaggle/input/petfinder-pawpularity-score/train.csv')
TEST = pd.read_csv('/kaggle/input/petfinder-pawpularity-score/test.csv')
SUB = pd.read_csv('/kaggle/input/petfinder-pawpularity-score/sample_submission.csv')

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision

from tqdm.notebook import tqdm

from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

import albumentations
import albumentations.pytorch

from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from sklearn.model_selection import KFold

from sklearn.metrics import f1_score

import cv2
from PIL import Image

In [ ]:
# sq = 10000000000
# min_width = 0
# min_height = 0
# min_path = ''
# scale = []
# from PIL import Image
# for i in tqdm(range(len(TRAIN['Id']))):
#     path = '../input/petfinder-pawpularity-score/train/'+ TRAIN['Id'][i] + '.jpg'
#     im = Image.open(path)
#     (width, height) = im.size
#     scale.append(width / height) 
#     if sq > width*height:
#         sq = width*height
#         min_width = width
#         min_height = height
#         min_path = path

# print(f'Plosh - {sq}' + ' | ' + f'Min_width {min_width}' + ' | ' + f'Min_height {min_height}')
# print(min_path)
# print(sum(scale) / len(scale))

In [ ]:
# x = cv2.imread('../input/dataset/train/train_images/1.jpg',cv2.IMREAD_COLOR)
img = Image.open('../input/petfinder-pawpularity-score/train/4388dabf50790924baa7fec88b192b02.jpg')
# x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
img = img.resize((380, 380), Image.ANTIALIAS)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import colors, cm, pyplot as plt

img=img 
plt.imshow(img)
plt.show()

In [ ]:
image = Image.open('../input/petfinder-pawpularity-score/train/4388dabf50790924baa7fec88b192b02.jpg')
image = image.resize((380, 380), Image.ANTIALIAS)

image = np.asarray(image)
# input_tensor[0] = input_tensor[0] / 255.0
# input_tensor[1] = input_tensor[1] / 255.0
# input_tensor[2] = input_tensor[2] / 255.0
preprocess = albumentations.Compose([
#     albumentations.CenterCrop(380, 380),
#     albumentations.Transpose(p=0.5),
#     albumentations.HorizontalFlip(p=0.5),
#     albumentations.VerticalFlip(p=0.5),
#     albumentations.ShiftScaleRotate(p=0.5),
    albumentations.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225],
    ),
    albumentations.pytorch.transforms.ToTensorV2()
])
input_tensor = preprocess(image = image)

plt.imshow(input_tensor['image'].numpy().transpose())
plt.show()

In [ ]:
TRAIN = pd.read_csv('../input/petfinder-pawpularity-score/train.csv')
TRAIN['path'] = TRAIN['Id'].apply(lambda x: 
                            str('../input/petfinder-pawpularity-score' 
                                + '/train/'
                                + str(x)
                               + '.jpg'))

TEST = pd.read_csv('../input/petfinder-pawpularity-score/test.csv')
TEST['path'] = TEST['Id'].apply(lambda x: 
                            str('../input/petfinder-pawpularity-score' 
                                + '/test/'
                                + str(x) 
                               + '.jpg'))

In [ ]:
from sklearn.model_selection import StratifiedKFold

In [ ]:
TRAIN = pd.concat((TRAIN, TRAIN.sample(8)), axis = 0).reset_index()

In [ ]:
class CustomImageDataset(Dataset):
    def __init__(self, annotations_file, img_path, transform=None, target_transform=None):
        self.img_labels = annotations_file
        self.img_path = img_path
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        
        image = Image.open(self.img_path[idx])
        
        (width, height) = image.size
           
        mult = 380 / min(width, height)

        new_width = int(round(mult*width,0))
        new_height = int(round(mult*height,0))

        image = image.resize((new_width, new_height), Image.ANTIALIAS)

        image_num = np.asarray(image)

        preprocess = albumentations.Compose([
            albumentations.CenterCrop(380, 380),
#             albumentations.Transpose(p=0.5),
#             albumentations.HorizontalFlip(p=0.5),
#             albumentations.VerticalFlip(p=0.5),
#             albumentations.ShiftScaleRotate(p=0.5),
#             albumentations.ColorJitter(brightness=0.2,
#                                        contrast=0.2,
#                                        saturation=0.2,
#                                        hue=0.2,
#                                        always_apply=False,
#                                        p=0.5),
            albumentations.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
            ),
            albumentations.pytorch.transforms.ToTensorV2()
        ])

        
        image = preprocess(image = image_num)
        
        label = self.img_labels[idx]
        label = torch.tensor(label)
            
        return image['image'], label 

In [ ]:
# for featur, datas, targ in test_dataloader:
#     print(featur.shape, datas.shape, targ.shape)
#     break

In [ ]:
# with torch.no_grad():
#     pred = model(featur.to(device = device), datas.to(device = device))

In [ ]:
# pred

In [ ]:
# criterion = torch.nn.MSELoss()
# torch.sqrt(criterion(pred, targ.to(device = device)))

In [ ]:
torch.cuda.empty_cache()

In [ ]:
import gc
gc.collect()

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

In [ ]:
import timm

In [ ]:
# class MyModel(nn.Module):
#     def __init__(self):
#         super(MyModel, self).__init__()
#         self.cnn = timm.create_model(model_name='tf_efficientnet_b4_ns', pretrained=True)
#         self.cnn.drop_rate = 0.4
#         self.cnn.classifier = nn.Sequential(
#             nn.Dropout(p=0.5, inplace=False),
#             nn.Linear(in_features = 1792, out_features = 30))    
        
#         self.fc1 = nn.Linear(30 + 12, 60)
#         self.fc2 = nn.Linear(60, 1)
        
#     def forward(self, image, data):
#         x1 = self.cnn(image)
#         x2 = data
        
#         x = torch.cat((x1, x2), dim=1)
#         x = nn.functional.relu(self.fc1(x))
#         x = self.fc2(x)
#         return x
        
model = timm.create_model(model_name='tf_efficientnet_b4_ns', pretrained=True)
model.drop_rate = 0.4
model.classifier = nn.Sequential(
            nn.Linear(in_features = 1792, out_features = 1))

# model.load_state_dict(torch.load('../input/pet-eff-b4/efficientnet_b4.pt'))
model.to(device = device)

In [ ]:
torch.cuda.empty_cache()

In [ ]:
def train_model(model, batch_size, n_epochs):
    
    best_result = {'epoch':0, 'valid_loss': 1000, 'train_loss': 1000}
    
    # to track the training loss as the model trains
    train_losses = []
    # to track the validation loss as the model trains
    valid_losses = []
    # to track the average training loss per epoch as the model trains
    avg_train_losses = []
    # to track the average validation loss per epoch as the model trains
    avg_valid_losses = [] 

    
    
    for epoch in tqdm(range(1, n_epochs + 1)):

        ###################
        # train the model #
        ###################
        wandb.watch(model)
        model.train() # prep model for training
        for batch, (image, target) in enumerate(train_dataloader, 1):
            # clear the gradients of all optimized variables
            image = image.to(device=device)
            target = target.to(device=device)
            
            optimizer.zero_grad()
            # forward pass: compute predicted outputs by passing inputs to the model
            output = model(image)
            # calculate the loss
            loss = criterion(output.squeeze(), target.float())
            loss = loss
            # backward pass: compute gradient of the loss with respect to model parameters
            loss.backward()
            # perform a single optimization step (parameter update)
            optimizer.step()
            # record training loss
            train_losses.append(loss.item())
            
            if batch % 2000 == 0:
                print(f'{batch / 100 * 100} batches done')
            
            del image
            del output
            del target
            torch.cuda.empty_cache()
            
        ######################    
        # validate the model #
        ######################
        model.eval() # prep model for evaluation

        for image, target in test_dataloader:
            image = image.to(device=device)
            target = target.to(device=device)
            # forward pass: compute predicted outputs by passing inputs to the model
            output = model(image)
            # calculate the loss
            loss = criterion(output.squeeze(), target.float())
            # record validation loss
            valid_losses.append(loss.item())
            

            del image
            del output
            del target
            torch.cuda.empty_cache()
        

        # print training/validation statistics 
        # calculate average loss over an epoch
        train_loss = np.average(train_losses)
        valid_loss = np.average(valid_losses)
#         f1_glob_f = np.average(f1_glob)
        avg_train_losses.append(train_loss)
        avg_valid_losses.append(valid_loss)
#         av_f1_glob.append(f1_glob_f)
        
        if best_result['valid_loss'] > valid_loss:
            best_result['valid_loss'] = valid_loss
            best_result['train_loss'] = train_loss
            best_result['epoch'] = epoch
            torch.save(model.state_dict(), 'checkpoint.pt')
        
        scheduler.step(valid_loss)
        
        epoch_len = len(str(n_epochs))
        
#         wandb.log(
#             {
#         "Train_loss": train_loss,
#         "Val Loss": valid_loss,
#         "Test Accuracy": 100. * correct / len(test_dataloader.dataset)
#             })
        print(
            f"Train_loss - {train_loss} | " + 
            f"Val Loss - {valid_loss}"
        )
        wandb.log({"train_loss": train_loss, 'valid_loss': valid_loss})
            # clear lists to track next epoch
        train_losses = []
        valid_losses = []
        
    model.load_state_dict(torch.load('checkpoint.pt'))

    return  model, avg_train_losses, avg_valid_losses, best_result

In [ ]:
config = wandb.config
config.learning_rate = 1e-4
config.weight_decay = 1e-6
config.n_epochs = 4
config.batch_size = 4
config.loss = 'MSELoss'
config.model = 'tf_efficientnet_b4_ns'
config.optim = 'ReduceLROnPlateau'
config.factor = 0.2
config.patience = 5
config.eps = 1e-6
config.CenterCrop = 380
config.Normalize_mean = 0.456
config.Normalize_std = 0.224

batch_size = 4
n_epochs = 4

X = TRAIN.drop(['Id','Pawpularity', 'index'], axis = 1)
y = TRAIN['Pawpularity']

skf = KFold(n_splits=5, shuffle=True)
skf.get_n_splits(X, y)

criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr= 1e-4, weight_decay = 1e-6)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.2, patience=5, eps=1e-6)

for train_index, test_index in skf.split(X, y):
    
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    

    train_loader = CustomImageDataset(y_train.values ,X_train['path'].values)
    test_loader = CustomImageDataset(y_test.values ,X_test['path'].values)

    train_dataloader = DataLoader(train_loader, 
                                  batch_size=batch_size, 
                                  shuffle=True, 
                                  num_workers = 4, 
                                  pin_memory=True)
    test_dataloader = DataLoader(test_loader, 
                                 batch_size=batch_size, 
                                 shuffle=True, 
                                 num_workers = 4, 
                                 pin_memory=True)



    model, train_loss, valid_loss, best_result = train_model(model,
                                                             batch_size = batch_size, 
                                                             n_epochs = n_epochs)
    del train_dataloader
    del test_dataloader
    del train_loader
    del test_loader
    torch.cuda.empty_cache()
    
    
    
# checkpoint = {'model': MyModel(),
#           'state_dict': model.state_dict(),
#           'optimizer' : optimizer.state_dict()}

# torch.save(checkpoint, 'Pet_eff_b4.pth')

In [ ]:
sdsd

In [ ]:
class CustomImageDataset_test(Dataset):
    def __init__(self, img_path, add_data, transform=None, target_transform=None):
        self.img_path = img_path
        self.add_data = add_data
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_path)

    def __getitem__(self, idx):
        
        image = Image.open(self.img_path[idx])
        
        (width, height) = image.size
           
        mult = 380 / min(width, height)

        new_width = int(round(mult*width,0))
        new_height = int(round(mult*height,0))

        image = image.resize((new_width, new_height), Image.ANTIALIAS)

        image_num = np.asarray(image)

        preprocess = albumentations.Compose([
            albumentations.CenterCrop(380, 380),
#             albumentations.Transpose(p=0.5),
#             albumentations.HorizontalFlip(p=0.5),
#             albumentations.VerticalFlip(p=0.5),
#             albumentations.ShiftScaleRotate(p=0.5),
#             albumentations.ColorJitter(brightness=0.2,
#                                        contrast=0.2,
#                                        saturation=0.2,
#                                        hue=0.2,
#                                        always_apply=False,
#                                        p=0.5),
            albumentations.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
            ),
            albumentations.pytorch.transforms.ToTensorV2()
        ])

        
        image = preprocess(image = image_num)
        
        add_data = self.add_data[idx]
        add_data = torch.tensor(add_data)
            
        return image['image'], add_data

In [ ]:
batch_size = 1    
    
test_loader = CustomImageDataset_test(TEST['path'].values, TEST.drop(['path', 'Id'], axis = 1).values)
test_dataloader = DataLoader(test_loader, batch_size=batch_size, num_workers = 4, pin_memory=True)

In [ ]:
model.eval()
    
final_targets = []
final_outputs = []
    
with torch.no_grad():
        
    for image, data in tqdm(test_dataloader, position=0, leave=True, desc='Evaluating'):
        
        image = image.to(device=device)
        data = data.to(device=device)
            
        output = model(image, data)
            
        final_outputs.extend(output.cpu().numpy().tolist())
        del image
        del output
        torch.cuda.empty_cache()

In [ ]:
res = final_outputs

In [ ]:
res

In [ ]:
submission = pd.DataFrame(res, columns = ['Pawpularity'])
submission = pd.concat([SUB['Id'], submission], axis=1) 
submission

In [ ]:
submission.to_csv('submission.csv', index=False)